In [6]:
!pip install nltk textblob

# First we will import the necessary Library
import datetime
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import os
import math

nltk.download('vader_lexicon')



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
# Parameters:
ndays = 30 #30 by default (start date: today -30 days, end date: today)
time_step = 60   # 60 minutes by default

# Sentiment Analysis


# Import News Dataset

In [8]:
'''
from google.colab import drive
import pandas as pd

# Monter Google Drive
drive.mount('/content/drive')



# Lire le fichier btc_data.csv
df = pd.read_csv('/content/drive/My Drive/Copie de news_datasets_text.csv')
'''
df = pd.read_csv('news_datasets_text.csv')

# NLTK

In [9]:

# NLTK
def apply_analysis(tweet):
    return SentimentIntensityAnalyzer().polarity_scores(tweet)

# Article Text
text = df
text[['neg','neu','pos','compound']] = df['Text'].apply(apply_analysis).apply(pd.Series)

def sentimental_analysis(df):
    if df['neg'] > df['pos']:
        return 'Negative'
    elif df['pos'] > df['neg']:
        return 'Positive'
    elif df['pos'] == df['neg']:
        return 'Neutral'


## Calculate mean of 'neg', 'pos'.. of all articles in time_step (X minutes)
# Calculate the mean 'neg.' grouped by date
mean_neg = text.groupby('date')['neg'].mean()
text = pd.merge(text, mean_neg, on='date', how='left', suffixes=('', '_mean'))
text['neg'] = text['neg_mean']
text.drop(columns=['neg_mean'], inplace=True)


# Calculate the mean 'pos' grouped by date
mean_pos = text.groupby('date')['pos'].mean()
text = pd.merge(text, mean_pos, on='date', how='left', suffixes=('', '_mean'))
text['pos'] = text['pos_mean']
text.drop(columns=['pos_mean'], inplace=True)

# Calculate the mean 'compound' grouped by date
mean_compound = text.groupby('date')['compound'].mean()
text = pd.merge(text, mean_compound, on='date', how='left', suffixes=('', '_mean'))
text['compound'] = text['compound_mean']
text.drop(columns=['compound_mean'], inplace=True)

# Calculate the mean 'neu' grouped by date
mean_neu = text.groupby('date')['neu'].mean()
text = pd.merge(text, mean_neu, on='date', how='left', suffixes=('', '_mean'))
text['neu'] = text['neu_mean']
text.drop(columns=['neu_mean'], inplace=True)


text['Sentiment_NLTK'] = text.apply(sentimental_analysis, axis = 1)

text.head()

,source,author,Title,description,url,urlToImage,date,Text,year,day,neg,neu,pos,compound,Sentiment_NLTK
0,"{'id': None, 'name': 'The Sports Daily'}",Gia Nguyen,Best Crypto Sportsbooks For Super Bowl 2024: B...,"Heading into Super Bowl 2024, Bitcoin is up ne...",https://thesportsdaily.com/news/best-crypto-sp...,https://thesportsdaily.com/wp-content/uploads/...,2024-02-11 03:00,"Heading into Super Bowl 2024, Bitcoin is up ne...",2024,11,0.042,0.859,0.099,0.4997,Positive
1,"{'id': None, 'name': 'Decrypt'}",Mathew Di Salvo,"This Week in Coins: Bitcoin and Solana Jump, R...",Both Bitcoin and Solana cleared 11% gains over...,https://decrypt.co/216781/this-week-in-coins-b...,https://cdn.decrypt.co/resize/1024/height/512/...,2024-02-11 00:00,Illustration by Mitchell Preffer for Decrypt.\...,2024,11,0.059,0.816,0.125,0.3818,Positive
2,"{'id': None, 'name': 'Biztoc.com'}",investorplace.com,Millionaire-Maker Altcoins: 3 Moonshot Cryptos...,Get in early on the early-stage crypto project...,https://biztoc.com/x/24e7c38331fcb375,https://c.biztoc.com/p/24e7c38331fcb375/s.webp,2024-02-11 23:00,Get in early on the early-stage crypto project...,2024,11,0.000,0.870,0.130,0.4939,Positive
3,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart CA Affiliate Program – Payout, Revie...","Welcome to the CoinSmart CA Affiliate Program,...",https://zacjohnson.com/coinsmart-ca-affiliate-...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11 17:00,"Welcome to the CoinSmart CA Affiliate Program,...",2024,11,0.000,0.735,0.265,0.8402,Positive
4,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart Affiliate Program – Payout, Review a...",Are you looking to monetize your website or so...,https://zacjohnson.com/coinsmart-affiliate-pro...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11 16:00,Are you looking to monetize your website or so...,2024,11,0.000,0.769,0.231,0.7964,Positive


# TextBlob

In [10]:
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity
def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity
def getSentiment(score):
    if score<0:
        return 'Negative'
    elif score==0:
        return 'Neutral'
    else:
        return 'Positive'

text['Subjectivity']=text['Text'].apply(getSubjectivity)
text['Polarity']=text['Text'].apply(getPolarity)


# Calculate the mean 'Polarity' of all articles in time_step (X minutes)
mean_polarity = text.groupby('date')['Polarity'].mean()
text = pd.merge(text, mean_polarity, on='date', how='left', suffixes=('', '_mean'))
text['Polarity'] = text['Polarity_mean']
text.drop(columns=['Polarity_mean'], inplace=True)



text['Sentiment_TB']=text['Polarity'].apply(getSentiment)
text.head()


,source,author,Title,description,url,urlToImage,date,Text,year,day,neg,neu,pos,compound,Sentiment_NLTK,Subjectivity,Polarity,Sentiment_TB
0,"{'id': None, 'name': 'The Sports Daily'}",Gia Nguyen,Best Crypto Sportsbooks For Super Bowl 2024: B...,"Heading into Super Bowl 2024, Bitcoin is up ne...",https://thesportsdaily.com/news/best-crypto-sp...,https://thesportsdaily.com/wp-content/uploads/...,2024-02-11 03:00,"Heading into Super Bowl 2024, Bitcoin is up ne...",2024,11,0.042,0.859,0.099,0.4997,Positive,0.454167,0.170833,Positive
1,"{'id': None, 'name': 'Decrypt'}",Mathew Di Salvo,"This Week in Coins: Bitcoin and Solana Jump, R...",Both Bitcoin and Solana cleared 11% gains over...,https://decrypt.co/216781/this-week-in-coins-b...,https://cdn.decrypt.co/resize/1024/height/512/...,2024-02-11 00:00,Illustration by Mitchell Preffer for Decrypt.\...,2024,11,0.059,0.816,0.125,0.3818,Positive,0.625000,-0.354167,Negative
2,"{'id': None, 'name': 'Biztoc.com'}",investorplace.com,Millionaire-Maker Altcoins: 3 Moonshot Cryptos...,Get in early on the early-stage crypto project...,https://biztoc.com/x/24e7c38331fcb375,https://c.biztoc.com/p/24e7c38331fcb375/s.webp,2024-02-11 23:00,Get in early on the early-stage crypto project...,2024,11,0.000,0.870,0.130,0.4939,Positive,0.433333,-0.050000,Negative
3,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart CA Affiliate Program – Payout, Revie...","Welcome to the CoinSmart CA Affiliate Program,...",https://zacjohnson.com/coinsmart-ca-affiliate-...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11 17:00,"Welcome to the CoinSmart CA Affiliate Program,...",2024,11,0.000,0.735,0.265,0.8402,Positive,0.866667,0.633333,Positive
4,"{'id': None, 'name': 'Zacjohnson.com'}",Disclaimer and DMCA,"CoinSmart Affiliate Program – Payout, Review a...",Are you looking to monetize your website or so...,https://zacjohnson.com/coinsmart-affiliate-pro...,https://0043d976.rocketcdn.me/wp-content/uploa...,2024-02-11 16:00,Are you looking to monetize your website or so...,2024,11,0.000,0.769,0.231,0.7964,Positive,0.322222,0.177778,Positive


# Data Cleaning - Final news_dataset

In [11]:
text_grouped = text.groupby('date')[['neg', 'neu', 'pos', 'compound', 'Subjectivity', 'Polarity']].mean()
text = text[['date',
       'Sentiment_NLTK','Sentiment_TB']]
text = pd.merge(text, text_grouped, on='date', how='inner')
text.drop_duplicates(subset='date', inplace=True)

# Fill missing time steps with neutral sentiment:

# Specify the time step
step = timedelta(minutes=time_step)


# Get the current date and time
current_datetime = datetime.now()
# Replace the time with midnight
midnight_datetime = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

# Define the date range
start_date = (midnight_datetime - timedelta(days=ndays)).strftime('%Y-%m-%d %H:%M')
end_date = midnight_datetime.strftime('%Y-%m-%d %H:%M')
date_range = pd.date_range(start=start_date, end=end_date, freq=step)

# Create a DataFrame with the complete date range
complete_dates_df = pd.DataFrame(date_range, columns=['date'])
text['date'] = pd.to_datetime(text['date'], format='%Y-%m-%d %H:%M')

# Merge your existing DataFrame with the complete date range DataFrame
merged_df = pd.merge(complete_dates_df, text, on='date', how='left')

# Fill missing values with defaults
merged_df['neg'].fillna(0, inplace=True)
merged_df['neu'].fillna(0, inplace=True)
merged_df['pos'].fillna(0, inplace=True)
merged_df['compound'].fillna(0, inplace=True)
merged_df['Subjectivity'].fillna(0, inplace=True)
merged_df['Polarity'].fillna(0, inplace=True)
merged_df['Sentiment_NLTK'].fillna('Neutral', inplace=True)
merged_df['Sentiment_TB'].fillna('Neutral', inplace=True)

text = merged_df

text

,date,Sentiment_NLTK,Sentiment_TB,neg,neu,pos,compound,Subjectivity,Polarity
0,2024-01-16 00:00:00,Neutral,Neutral,0.000,1.000,0.000,0.0000,1.000000,0.00000
1,2024-01-16 01:00:00,Positive,Positive,0.000,0.945,0.055,0.2023,0.443750,0.06875
2,2024-01-16 02:00:00,Neutral,Neutral,0.000,0.000,0.000,0.0000,0.000000,0.00000
3,2024-01-16 03:00:00,Positive,Negative,0.069,0.813,0.117,0.1531,0.758333,-0.17500
4,2024-01-16 04:00:00,Neutral,Neutral,0.000,0.000,0.000,0.0000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...
716,2024-02-14 20:00:00,Neutral,Neutral,0.000,0.000,0.000,0.0000,0.000000,0.00000
717,2024-02-14 21:00:00,Neutral,Neutral,0.000,0.000,0.000,0.0000,0.000000,0.00000
718,2024-02-14 22:00:00,Neutral,Neutral,0.000,0.000,0.000,0.0000,0.000000,0.00000
719,2024-02-14 23:00:00,Neutral,Neutral,0.000,0.000,0.000,0.0000,0.000000,0.00000


Save dataframe as CSV

In [12]:
text.to_csv('news_sentiment.csv', encoding='utf-8-sig',index= False)
print('Done')

Done
